In [ ]:
# -*- coding: utf-8 -*-
"""
    Convert wav to spectrograms
"""

# import numpy as np
# from array import array

import os
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as pltb

In [ ]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri May 14 12:15:44 2021

SoundFile class
"""

# https://librosa.org/doc/latest/search.html?q=export&check_keywords=yes&area=default#
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import skimage.io
import os
from pathlib import Path

from joblib import dump

def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled


class SoundFile:
    """ 
    ----------
    SoundFile class contains informations about one sound related to a wav file

    ----------
    Parameters:
        name_file : full path + name of the file ('./fan/train/normal_id_00_00000000.wav')
        out_folder_png: name of output folder in order to save the spectograms as pngs
    """
    # settings
    # hop_length = 512 # number of samples per time-step in spectrogram
    # n_mels = 64 # number of bins in spectrogram. Height of image
    # time_steps = 384 # number of time-steps. Width of image

    
    def __init__(self, nameFile, out_folder_png, image_width=0):
        #print('init the audio file:', nameFile)
        self.nameFile = nameFile
        self.out_folder_png = out_folder_png
        self.samples, self.sample_rate = librosa.load(nameFile, sr=None)
        # self.samples : (160000,) complex matrix
        # self.sample_rate : 16000

        if image_width > 0:
          # to have fixed width image
          hop_length = int(self.samples.shape[0]/image_width)
          #print("hop_length : ", hop_length)

          self.sgram = librosa.stft(self.samples, hop_length=hop_length+1)
        else:
          self.sgram = librosa.stft(self.samples)

        #print("self.sgram.shape", self.sgram.shape)
        # self.sgram.shape : (1025, 313) matrix
        # (f,t) => f = frequency, t = time
        #print(self.sgram)
        # n_fft=2000 => (1001, 321)
        # n_fft=500 => (251, 1281)
        # n_fft=32 => (17, 200001)
        
    def exportMelSpectrogram(self, image_height=128): # black and white with skimage
        # mels = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels,
        #                                     n_fft=hop_length*2, hop_length=hop_length)
        
        p = Path(self.nameFile)
        
        sgram_mag, _ = librosa.magphase(self.sgram)
        mels = librosa.feature.melspectrogram(S=sgram_mag, sr=self.sample_rate, n_mels=image_height)
        # mel_sgram = librosa.amplitude_to_db(mel_scale_sgram, ref=np.min)

        # save raw matrix
        nameMels = self.out_folder_png + p.stem + '_mels.joblib'
        dump(mels, nameMels, compress=True)

        mels = np.log(mels + 1e-9) # add small number to avoid log(0)
    
        #print("mels.shape :", mels.shape) # (128, 313)
        # min-max scale to fit inside 8-bit range
        img = scale_minmax(mels, 0, 255).astype(np.uint8)
        img = np.flip(img, axis=0) # put low frequencies at the bottom in image
        img = 255-img # invert. make black==more energy
    
        # save as PNG in out_folder_png
        # namePng = os.path.splitext(self.nameFile)[0] + '.png'
        namePng = self.out_folder_png + p.stem + '.png' #same name for the png file as wav
        skimage.io.imsave(namePng, img)

        
        

    def showMelSpectrogram(self):
         # use the decibel scale to get the final Mel Spectrogram - v2
        sgram_mag, _ = librosa.magphase(self.sgram)
        mel_scale_sgram = librosa.feature.melspectrogram(S=sgram_mag, sr=self.sample_rate)
        mel_sgram = librosa.amplitude_to_db(mel_scale_sgram, ref=np.min)
        librosa.display.specshow(mel_sgram, sr=self.sample_rate, x_axis='time', y_axis='mel')
        # plt.colorbar(format='%+2.0f dB'); # vertical legend



In [ ]:
def convertSound2ImageDir(soundFolder, imageFolder):

    if not os.path.exists(imageFolder):
        os.makedirs(imageFolder)

    wavfiles = [f for f in listdir(soundFolder) if isfile(join(soundFolder, f))]
    # wavfiles.remove('.DS_Store')
    for f in wavfiles:
        if f[-4:] != '.wav':
            # ignore non .wav files
            continue

        print(f)
        s = SoundFile(soundFolder + f, imageFolder)
        s.exportMelSpectrogram()
        # s.showMelSpectrogram()
        # plt.show()



In [ ]:
def convertSound2ImageDirV2(soundFolder, imageFolder, width=128, height=64):

    if not os.path.exists(imageFolder):
        os.makedirs(imageFolder)

    wavfiles = [f for f in listdir(soundFolder) if isfile(join(soundFolder, f))]
    # wavfiles.remove('.DS_Store')
    for f in wavfiles:
        if f[-4:] != '.wav':
            # ignore non .wav files
            continue

        print(f)
        s = SoundFile(soundFolder + f, imageFolder, image_width=width)
        s.exportMelSpectrogram(image_height=height)
        # s.showMelSpectrogram()
        # plt.show()



In [ ]:
# convert to images of automatic size
#machines = ['fan', 'pump', 'slider', 'ToyCar', 'ToyConveyor', 'valve']
machines = ['fan']
sets = ['train', 'test']

dataFolder = '/drive/MyDrive/asdpy/data/original/dc2020task2'

for machine in machines:
    for s in sets:
        # example : '../data/fan/train/'
        soundFolder = dataFolder+'/'+machine+'/'+s+'/'

        # example : '../data/fan/train_png/'
        imageFolder = dataFolder+'/'+machine+'/'+s+'_png/'

        print("Convert directory ", soundFolder)

        convertSound2ImageDir(soundFolder, imageFolder)


Convert directory  /drive/MyDrive/asdpy/data/original/dc2020task2/fan/train/
normal_id_04_00000865.wav


UnboundLocalError: ignored

In [ ]:
# convert to images of fixed size
#machines = ['fan', 'pump', 'slider', 'ToyCar', 'ToyConveyor', 'valve']
machines = ['slider', 'ToyCar', 'ToyConveyor', 'valve']
sets = ['train', 'test']

dataFolder = '/drive/MyDrive/asdpy/data/original/dc2020task2'

for machine in machines:
    for s in sets:
        # example : '../data/fan/train/'
        soundFolder = dataFolder+'/'+machine+'/'+s+'/'

        # example : '../data/fan/train_png/'
        imageFolder = dataFolder+'/'+machine+'/'+s+'_png_128_64/'

        print("Convert directory ", soundFolder)

        convertSound2ImageDirV2(soundFolder, imageFolder, width=128, height=64)


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
normal_id_01_00000068.wav
normal_id_01_00000094.wav
normal_id_01_00000071.wav
normal_id_01_00000088.wav
normal_id_01_00000100.wav
normal_id_01_00000052.wav
normal_id_01_00000082.wav
normal_id_01_00000073.wav
normal_id_01_00000105.wav
normal_id_01_00000090.wav
normal_id_01_00000077.wav
normal_id_01_00000057.wav
normal_id_01_00000080.wav
normal_id_01_00000070.wav
normal_id_01_00000051.wav
normal_id_01_00000064.wav
normal_id_01_00000089.wav
normal_id_01_00000056.wav
normal_id_01_00000065.wav
normal_id_01_00000075.wav
normal_id_01_00000083.wav
normal_id_01_00000072.wav
normal_id_01_00000095.wav
normal_id_01_00000074.wav
normal_id_01_00000103.wav
normal_id_01_00000066.wav
normal_id_01_00000081.wav
normal_id_01_00000078.wav
normal_id_01_00000139.wav
normal_id_01_00000122.wav
normal_id_01_00000108.wav
normal_id_01_00000137.wav
normal_id_01_00000161.wav
normal_id_01_00000119.wav
normal_id_01_00000154.wav
normal_id_01